In [19]:
import torch
from torch import nn, optim
import pandas as pd
from res.plot_lib import plot_data, plot_model, set_default
import numpy as np
import torchvision.transforms as T
import torch.nn.functional as F

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
train = pd.read_csv('~/Desktop/Train.csv')

train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
test = pd.read_csv('~/Desktop/Test.csv')

test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
labal = train[train.columns[:1]]
labal.head()

,label
0,2
1,9
2,6
3,0
4,3


In [24]:
features = train[train.columns[1:]]
features.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,5,0,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,1,2,0,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
y = labal.to_numpy()
x = features.to_numpy()
y = torch.from_numpy(y)
x = torch.from_numpy(x)

In [26]:
x = torch.reshape(x, (60000, 28, 28))

In [9]:
x

torch.Size([60000, 28, 28])

In [10]:
y

tensor([[2],
        [9],
        [6],
        ...,
        [8],
        [8],
        [7]])

In [18]:
train_loader = torch.utils.data.DataLoader(x, batch_sampler = 50)

In [15]:
class convnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1)
        self.conv2 = nn.Conv2d(1, 1, 3, 1)
        self.fc1 = nn.Linear(784, 392)
        self.fc2 = nn.Linear(392, 392)
        self.fc3 = nn.Linear(392, 9)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.relu(self.fc1)
        x = self.fc2(x)
        x = F.relu(self.fc1)
        x = self.fc3(x)
        return F.logsoftmax(x, dim=1)
    
#nn.conv2d(The number of input images, the number of output images, the size of like image that takes the dot product or cross product(i forgot) and the shift of the kernal

In [16]:
model = convnet()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()